In [19]:
import os
import shutil
from IPython.display import clear_output
from timeit import default_timer as timer
from tqdm import tqdm
import datetime
# pip install mysql-connector
import sqlite3
import pandas as pd

In [28]:
class DTU:
    def __init__(self):
        self.source=""
        self.destination=""
        self.mediafolder="D:\\sourceDir"
        self.drives=os.listdir(self.mediafolder)
        #self.destFolder="USB Transfer"
    # use once to create tables
    def createTables(self):
        # id , time taken to process,amount of data,no of folders,
        sql = """CREATE TABLE datadtu (
            ID INTEGER PRIMARY KEY AUTOINCREMENT  ,
            nooffolders INTEGER(5),
            amountofdata FLOAT(11,11),
            processtime FLOAT(11,11)
        );"""
        self.getDatabaseInstance().execute(sql)
    # connect to database dtu.db and set cursor
    def connectDatabase(self):
        self.connection = sqlite3.connect("dtu.db")
        self.cursor = self.connection.cursor()
        
    # get instance of cursor (or get cursor object)
    def getDatabaseInstance(self):
        return self.cursor
    # make any Query
    def makeQuery(self,query):
        return self.cursor.execute(query)
    def showMenu(self):
        print("-----MENU-----")
        for i in range(1,7):
            if i == 1:
                print("[1] Show all sources\n")

            elif i== 2:
                print("[2] Select source\n")

            elif i== 3:
                print("[3] Select Destination\n")

            elif i== 4:
                print("[4] Transfer content\n")
            
            elif i== 5:
                print("[5] Print database\n")

            elif i== 6:
                print("[6] Exit\n")

            else:
                break

        choice = int(input("Please select an option: "))
        print("--------------")
        return choice
    
    def choiceSelected(self,choice):
        try:
            
            #Show all sources
            if choice == 1:
                print("Availabe sources:\n")
                for i,source in enumerate(self.drives):
                    print(i+1,source)
                return

            #Select source
            elif choice == 2:
                print("Availabe sources:\n")
                for i,source in enumerate(self.drives):
                    print(i+1,source)

                select = self.askChoice("Select a source: ")
                self.source=self.drives[select-1]
                print("\nSource selected!\n")
                return

            #Select dest
            elif choice == 3:
                print("Availabe destinations:\n")
                for i,source in enumerate(self.drives):
                    if source==self.source:
                        continue
                    print(i+1,source)

                select = self.askChoice("Select a destination: ")
                self.destination=self.drives[select-1]
                print("\nDestination selected!\n")
                return

            #Transfer content
            elif choice == 4:
                print("[1] Transfer all content\n","\n[2] Select content to transfer")
                ch=self.askChoice("\nChoose an option: ")
                if ch == 1:
                    start = timer()
                    files = os.listdir(os.path.join(self.mediafolder,self.source))
                    filelen = len(files)
                    destinationDir = os.path.join(self.mediafolder,self.destination)
                    sourceDir = os.path.join(self.mediafolder,self.source)
                    # paste commented code here
                    for f in tqdm(range(filelen),unit='KB'):
                        if os.path.isdir(os.path.join(sourceDir,files[f])):
                            shutil.copytree(os.path.join(sourceDir,files[f]),os.path.join(destinationDir,str(files[f])))
                    end = timer()  
                    dirsize = self.folderSize('D:\sourceDir\SOURCE')
                    i = 1
                    insertQuery = """INSERT INTO datadtu (nooffolders,amountofdata,processtime) VALUES (?,?,?);"""
                    self.cursor.execute(insertQuery,(filelen,dirsize,end))
                    self.connection.commit()
                    print("Time-Elapsed : " ,end - start,"sec.")

                elif ch == 2:
                    pass
                
                return
            
            elif choice == 5:
                self.printTable()
                
            elif choice == 6:
                pass 

        except Exception as e:
            print("ERROR: ",e)
            
    # Returns folder size (Recursive algorithm)
    def folderSize(self,path):
        total = 0
        for i in os.scandir(path):
            if i.is_file():
                total += i.stat().st_size
            elif i.is_dir():
                total += self.folderSize(i.path)
        return (total)
    def askChoice(self,string):
        return int(input(string))
    # prints database table
    def printTable(self):
        self.cursor.execute("""SELECT * FROM datadtu;""")
        result = self.cursor.fetchall()
        for row in result:
            print(row)
    # Convert Database into csv             
    def csvdb(self):
        df = pd.read_sql_query("SELECT * FROM datadtu;",self.connection)
        return df
if __name__ == "__main__":
    print("Welcome to DTU")
    dtu=DTU()
    dtu.connectDatabase()
    # un-comment this to make table in the database
    #dtu.createTables()
    
    while(True):
        ch = dtu.showMenu()

        #Exit condition
        if ch not in [1,2,3,4,5]:
            print("Bye for now!")
            break

        #On choice selected
        dtu.choiceSelected(ch)
        
        
        input("press [ENTER] to continue")
        clear_output()

-----MENU-----
[1] Show all sources

[2] Select source

[3] Select Destination

[4] Transfer content

[5] Print databse

[6] Exit

Please select an option: 6
--------------
Bye for now!
